In [ ]:
using Revise
using BenchmarkTools

In [ ]:
using DataFrames

using DataIO

In [ ]:
element_, score_ = eachcol(read_dataframe("/Users/kwat/Downloads/score_and_set/ccle_proteinAHR/gene_x_score.tsv"))

;

In [ ]:
using JSON

In [ ]:
json_path = "/Users/kwat/Downloads/score_and_set/ccle_proteinAHR/gene_set.json"

json_dict = JSON.parse(open(json_path))

json_dict = convert(Dict{String, Vector{String}}, json_dict)

In [ ]:
using GCTGMT

In [ ]:
gmt_path_ = map(gmt_path -> replace(gmt_path, ".." => "/Users/kwat/Downloads/"), json_dict["gene_sets_tested"])

set_to_element_ = read_gmt(gmt_path_)

;

In [ ]:
using FeatureSetEnrichment

In [ ]:
set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

;

In [ ]:
set_ = collect(keys(set_to_result))

set_score_ = collect(map(t -> t[3], values(set_to_result)))

;

In [ ]:
using Support

In [ ]:
set_score_, set_ = sort_like((set_score_, set_))

n = length(set_)

;

In [ ]:
for set in json_dict["gene_sets_positive"]
    
    println(set)
    
    i = findfirst(set_ .== set)

    println("Rank $i / $n")
    
#     score_set(element_, score_, set_to_element_[set]; title_text = set)
    
    score_set_new(element_, score_, set_to_element_[set])
    
end